In [2]:
import csv
import smtplib
import hashlib
import os
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email import encoders
import qrcode
from PIL import Image, ImageDraw, ImageFont
import configparser

# Read the configuration file
config = configparser.ConfigParser()
config.read('config.ini')

# SMTP server configuration
smtp_server = config.get('SMTP', 'server')
smtp_port = config.getint('SMTP', 'port')
smtp_username = config.get('SMTP', 'username')
smtp_password = config.get('SMTP', 'password')
smtp_from = config.get('SMTP', 'from')

# Rest of your code...

# Load the list of already processed ticket IDs
try:
    with open('processed_tickets.csv', 'r') as f:
        processed_tickets = f.read().splitlines()
except FileNotFoundError:
    processed_tickets = []

# Open the CSV file
with open('tickets.csv', 'r') as f:
    reader = csv.reader(f)
    next(reader)  # Skip the header row

    # For each row in the CSV file
    for row in reader:
        # Extract the ticket ID and recipient email
        ticket_id = row[1]  # The 10-character alphanumeric string
        recipient_email = row[7]

        # Calculate the SHA256 hash of the ticket ID
        ticket_id_hash = hashlib.sha256(ticket_id.encode()).hexdigest()

        # If we've already processed this ticket ID, skip
        if ticket_id_hash in processed_tickets:
            continue

        # Generate a QR code for the ticket ID
        qr = qrcode.QRCode(
            version=1,
            error_correction=qrcode.constants.ERROR_CORRECT_L,
            box_size=10,
            border=4,
        )
        qr.add_data(ticket_id)
        qr.make(fit=True)

        img = qr.make_image(fill='black', back_color='white')
        img = img.convert("RGB")

        # Create a new image with extra space for text
        new_img = Image.new("RGB", (img.width, img.height + 30))

        # Paste the QR code into the new image
        new_img.paste(img, (0, 0))

        # Add text under the QR code
        draw = ImageDraw.Draw(new_img)
        font = ImageFont.load_default()  # Use the default font
        draw.text((15, img.height + 15), ticket_id, font=font, fill=(255, 0, 0))  # Change text color to red

        filename = f'ticket_{ticket_id}.png'
        new_img.save(filename)

        # Create a multipart message
        msg = MIMEMultipart()
        msg['From'] = smtp_from
        msg['To'] = recipient_email
        msg['Subject'] = 'Your QR code'

        # Attach the QR code image to email
        with open(filename, 'rb') as f:
            part = MIMEBase('application', 'octet-stream')
            part.set_payload(f.read())
            encoders.encode_base64(part)
            part.add_header('Content-Disposition', f'attachment; filename={filename}')
            msg.attach(part)

        # Connect to the SMTP server and send the email
        with smtplib.SMTP_SSL(smtp_server, smtp_port) as server:
            server.login(smtp_username, smtp_password)
            server.send_message(msg)

        print(f'Email sent for ticket ID {ticket_id}')

        # Delete the QR code image
        os.remove(filename)

        # Add the processed ticket ID hash to the list of processed tickets
        processed_tickets.append(ticket_id_hash)

# Save the updated list of processed tickets
with open('processed_tickets.csv', 'w') as f:
    for ticket in processed_tickets:
        f.write(f'{ticket}\n')

In [3]:

def invalidate_ticket(ticket_id):
    # Calculate the SHA256 hash of the ticket ID
    ticket_id_hash = hashlib.sha256(ticket_id.encode()).hexdigest()

    # Load the list of already processed ticket IDs
    try:
        with open('processed_tickets.csv', 'r') as f:
            processed_tickets = f.read().splitlines()
    except FileNotFoundError:
        processed_tickets = []

    # If the ticket ID hash is in the list, remove it
    if ticket_id_hash in processed_tickets:
        processed_tickets.remove(ticket_id_hash)

    # Save the updated list of processed tickets
    with open('processed_tickets.csv', 'w') as f:
        for ticket in processed_tickets:
            f.write(f'{ticket}\n')

    print(f'Ticket ID {ticket_id} invalidated.')
invalidate_ticket(input('Bitte Ticket eingeben'))

Ticket ID  invalidated.


In [14]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
import configparser

import configparser
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time

try:
    # Read the credentials from the config file
    config = configparser.ConfigParser()
    config.read('config.ini')
    username = config.get('credentials', 'username')
    password = config.get('credentials', 'password')

    # Create a new instance of the Chrome driver
    driver = webdriver.Chrome()

    # Go to the specified URL
    driver.get("https://www.mk-waldburg-hannober.de/wp-admin/edit.php?post_type=tribe_events&page=tickets-attendees&event_id=2419")

    # Find the username field, enter the username, and press Enter
    username_field = driver.find_element(By.XPATH, '//*[@id="user_login"]')
    username_field.send_keys(username)

    # Find the password field, enter the password, and press Enter
    password_field = driver.find_element(By.XPATH, '//*[@id="user_pass"]')
    password_field.send_keys(password)
    password_field.send_keys(Keys.RETURN)

    # Click on the specified element
    element_to_click = driver.find_element(By.XPATH, '//*[@id="event-tickets__attendees-admin-form"]/div[1]/div[2]/a[1]')
    element_to_click.click()

    # Wait for 5 seconds
    time.sleep(5)

    # Logout
    driver.get("https://www.mk-waldburg-hannober.de/wp-login.php?action=logout&_wpnonce=")
    logout_button = driver.find_element(By.XPATH, '//*[@id="error-page"]/div/p[2]/a')
    logout_button.click()

    # Close the driver
    driver.quit()

except Exception as e:
    # Close the driver when Error occured
    driver.quit()
    print(f"An error occurred: {str(e)}")